# QuickStart

In this Notebook we'll run Archai's [Quickstart](https://microsoft.github.io/archai/getting_started/quick_start.html) example on Azure Machine Learning.

## Prerequisites

- Python 3.7 or later
- An Azure subscription.
- An Azure Resource Group
- An Azure Machine Learning [Workspace](https://learn.microsoft.com/en-us/azure/machine-learning/quickstart-create-resources#create-the-workspace).

## Requirements

- Install Azure SDK requirements

    `pip install azure-ai-ml azure-identity`

## Get a handle to the workspace

We load the workspace from a workspace [configuration file](https://learn.microsoft.com/en-us/azure/machine-learning/how-to-configure-environment#local-and-dsvm-only-create-a-workspace-configuration-file).

In [ ]:
from pathlib import Path
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential

# Set the path to your config. file
path = Path("../.azureml/config.json")

credential = DefaultAzureCredential()

ml_client = MLClient.from_config(
    credential=credential,
    path=path
)

## Create a compute cluster

We provision a Linux [compute cluster](https://learn.microsoft.com/en-us/azure/machine-learning/how-to-create-attach-compute-cluster?tabs=python) fos this Notebook. See the [full list](https://azure.microsoft.com/en-ca/pricing/details/machine-learning/) on VM sizes and prices.

In [ ]:
from azure.ai.ml.entities import AmlCompute

# Name assigned to the compute cluster
cpu_compute_target = "nas-cpu-cluster-D14-v2"

try:
    # Check if the compute target already exists
    cpu_cluster = ml_client.compute.get(cpu_compute_target)
    print(
        f"You already have a cluster named {cpu_compute_target}, we'll reuse it as is."
    )

except Exception:
    cpu_cluster = AmlCompute(
        name=cpu_compute_target,
        # Azure ML Compute is the on-demand VM service
        type="amlcompute",
        # VM Family
        size="Standard_D14_v2",
        # Minimum running nodes when there is no job running
        min_instances=0,
        # Nodes in cluster
        max_instances=4,
        # How many seconds will the node running after the job termination
        idle_time_before_scale_down=180,
        # Dedicated or LowPriority. The latter is cheaper but there is a chance of job termination
        tier="Dedicated",
    )

    cpu_cluster = ml_client.compute.begin_create_or_update(cpu_cluster)

print(
    f"AMLCompute with name {cpu_cluster.name} is created, the compute size is {cpu_cluster.size}"
)

## Create an environment based on a YAML file

In [ ]:
from azure.ai.ml.entities import Environment

custom_env_name = "aml-archai"

archai_job_env = Environment(
    name=custom_env_name,
    description="Custom environment for Archai Job",
    tags={"archai": "1.0.0"},
    conda_file="conda.yaml",
    image="mcr.microsoft.com/azureml/openmpi3.1.2-ubuntu18.04:latest",
    version="0.1.0"
)
archai_job_env = ml_client.environments.create_or_update(archai_job_env)

print(
    f"Environment with name {archai_job_env.name} is registered to workspace, the environment version is {archai_job_env.version}"
)

## Create job

In [ ]:
from azure.ai.ml import command
from azure.ai.ml import Output

job = command(
    display_name="Archai's QuickStart",
    outputs=dict(
        output_path=Output(type="uri_folder", mode="rw_mount")
    ),
    code="main.py",
    command="python main.py --output_dir ${{outputs.output_path}}",
    environment=f"{archai_job_env.name}:{archai_job_env.version}",
    compute=cpu_compute_target,
    experiment_name="archai_quickstart"
)

## Run job

In [ ]:
ml_client.create_or_update(job)